# Managing an Azure Kubernetes Service cluster from Cloud Shell

So we set up our AKS cluser, let's go ahead and run the import setup code from the last notbook and then do some troubleshooting work.

In [ ]:
Enter-AzShell

In [2]:
Set-AzContext -SubscriptionName 'Oaas-SubLib-013' | Out-Null
Import-AzAksCredential -ResourceGroupName manning-test-aks -Name manning-test-aks -Force

Set-AzContext -SubscriptionName 'Oaas-SubLib-013' | Out-Null


Import-AzAksCredential -ResourceGroupName manning-test-aks -Name manning-test-aks -Force


## Kubernetes Troubleshooting

> NOTE: This text was shamelessly stolen from [this set of docs by a Microsoft engineer](https://github.com/feiskyer/kubernetes-handbook/blob/master/en/troubleshooting/index.md).

This part introduces how to troubleshoot various problems on Kubernetes, includes

- [Troubleshooting Clusters](cluster.md)
- [Troubleshooting Pods](pod.md)
- [Troubleshooting Networking](network.md)
- [Troubleshooting Persistent Volumes](pv.md)
  - [AzureDisk](azuredisk.md)
  - [AzureFile](azurefile.md)
- [Troubleshooting Windows Containers](windows.md)
- [Troubleshooting Cloud Provider](cloud.md)
  - [Azure](azure.md)
- [Tools for Troubleshooting](tools.md)

Remember `kubectl` is always the most important tool when starting to troubleshoot any problems.

### Listing Nodes

In [16]:
kubectl get nodes

$nodename = kubectl get nodes -o=jsonpath='{.items[0].metadata.name}'
kubectl describe node $nodename

kubectl get nodes


NAME                                STATUS   ROLES   AGE   VERSION


aks-agentpool-22729220-vmss000000   Ready    agent   61m   v1.15.11


aks-agentpool-22729220-vmss000001   Ready    agent   60m   v1.15.11


aks-agentpool-22729220-vmss000002   Ready    agent   60m   v1.15.11


...

  attachable-volumes-azure-disk  0            0


Events:


  Type    Reason    Age   From                                           Message


  ----    ------    ----  ----                                           -------


  Normal  Starting  60m   kube-proxy, aks-agentpool-22729220-vmss000000  Starting kube-proxy.


### Listing Pods

In [25]:
kubectl get pods -o wide
kubectl -n kube-system get pods -o wide

kubectl get pods -o wide


NAME                               READY   STATUS    RESTARTS   AGE   IP           NODE                                NOMINATED NODE   READINESS GATES


nginx-deployment-8df4655b6-c4hgn   1/1     Running   0          60m   10.244.2.3   aks-agentpool-22729220-vmss000001   <none>           <none>


nginx-deployment-8df4655b6-sxm55   1/1     Running   0          60m   10.244.1.4   aks-agentpool-22729220-vmss000002   <none>           <none>


kubectl -n kube-system get pods -o wide


...

metrics-server-7d654ddc8b-zph6c         1/1     Running   0          76m   10.244.0.8   aks-agentpool-22729220-vmss000000   <none>           <none>


omsagent-6w9cz                          1/1     Running   0          75m   10.244.0.2   aks-agentpool-22729220-vmss000000   <none>           <none>


omsagent-hzw5b                          1/1     Running   1          74m   10.244.2.2   aks-agentpool-22729220-vmss000001   <none>           <none>


omsagent-rs-699dbb6cb8-pwr4c            1/1     Running   0          76m   10.244.0.6   aks-agentpool-22729220-vmss000000   <none>           <none>


omsagent-t9dc7                          1/1     Running   0          74m   10.244.1.2   aks-agentpool-22729220-vmss000002   <none>           <none>


### Looking at Pod events

In [19]:
$podname = kubectl get pods -o=jsonpath='{.items[0].metadata.name}'
kubectl describe pod $podname

$podname = kubectl get pods -o=jsonpath='{.items[0].metadata.name}'


kubectl describe pod $podname


Name:           nginx-deployment-8df4655b6-c4hgn


Namespace:      default


Priority:       0


...

  Normal  Scheduled  49m   default-scheduler                           Successfully assigned default/nginx-deployment-8df4655b6-c4hgn to aks-agentpool-22729220-vmss000001


  Normal  Pulling    49m   kubelet, aks-agentpool-22729220-vmss000001  Pulling image "nginx"


  Normal  Pulled     49m   kubelet, aks-agentpool-22729220-vmss000001  Successfully pulled image "nginx"


  Normal  Created    48m   kubelet, aks-agentpool-22729220-vmss000001  Created container nginx


  Normal  Started    48m   kubelet, aks-agentpool-22729220-vmss000001  Started container nginx


### Looking at kube-apiserver logs

In [22]:
# Looking at kube-apiserver logs
$apiserverPodname = kubectl -n kube-system get pod -l component=kube-apiserver -o jsonpath='{.items[0].metadata.name}'
if ($apiserverPodname) {
    kubectl -n kube-system logs $apiserverPodname --tail 100
}

# Looking at kube-controller-manager logs
$controllerPodname = kubectl -n kube-system get pod -l component=kube-controller-manager -o jsonpath='{.items[0].metadata.name}'
if ($controllerPodname) {
    kubectl -n kube-system logs $controllerPodname --tail 100
}

# Looking at kube-scheduler logs
$schedulerPodname = kubectl -n kube-system get pod -l component=kube-scheduler -o jsonpath='{.items[0].metadata.name}'
if ($schedulerPodname) {
    kubectl -n kube-system logs $schedulerPodname --tail 100
}

# Looking at kube-dns logs
$dnsPodname = kubectl -n kube-system get pod -l k8s-app=kube-dns -o jsonpath='{.items[0].metadata.name}'
if ($dnsPodname) {
    kubectl -n kube-system logs $dnsPodname -c kubedns
}


$apiserverPodname = kubectl -n kube-system get pod -l component=kube-apiserver -o jsonpath='{.items[0].metadata.name}'


error: error executing jsonpath "{.items[0].metadata.name}": Error executing template: array index out of bounds: index 0, length 0. Printing more information for debugging the template:


	template was:


		{.items[0].metadata.name}


	object given to jsonpath engine was:


...

if ($dnsPodname) {


>>     kubectl -n kube-system logs $dnsPodname -c kubedns


>> }


>> 


Error from server (BadRequest): container kubedns is not valid for pod coredns-698c77c5d7-gbdq4


### Looking at kube-proxy logs

In [24]:
kubectl -n kube-system get pod -l component=kube-proxy

$podname = kubectl -n kube-system get pod -l component=kube-proxy -o=jsonpath='{.items[0].metadata.name}'
kubectl -n kube-system logs $podname

kubectl -n kube-system get pod -l component=kube-proxy


NAME               READY   STATUS    RESTARTS   AGE


kube-proxy-bxbzh   1/1     Running   0          72m


kube-proxy-qv468   1/1     Running   0          73m


kube-proxy-vx5zv   1/1     Running   0          73m


...

I0614 05:12:03.015332       1 bounded_frequency_runner.go:221] sync-runner: ran, next possible in 0s, periodic in 30s


I0614 05:12:33.015636       1 proxier.go:701] Syncing iptables rules


I0614 05:12:33.045120       1 bounded_frequency_runner.go:221] sync-runner: ran, next possible in 0s, periodic in 30s


I0614 05:13:03.045351       1 proxier.go:701] Syncing iptables rules


I0614 05:13:03.072838       1 bounded_frequency_runner.go:221] sync-runner: ran, next possible in 0s, periodic in 30s
